In [232]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy


In [233]:
from matplotlib import pyplot as plt
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [234]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, ReLU, PReLU, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers, models, regularizers, constraints

In [235]:
x_train = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy")
x_test = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy")
y_train_coarse = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy")

In [236]:
from sklearn.model_selection import train_test_split

In [237]:
import pickle
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [238]:
y_train_fine.shape

(50000,)

In [239]:
x_train.shape

(50000, 32, 32, 3)

In [240]:
x_train,x_val, y_train, y_val = train_test_split(x_train, y_train_fine, test_size=0.1, stratify=y_train_fine)

In [241]:
y_val.shape

(5000,)

In [242]:
x_val.shape

(5000, 32, 32, 3)

In [243]:
x_train_horizontal_flip= x_train[:,:,::-1,:]
#x_train_vertical_flip = x_train[:,::-1,:,:]
#x_train_vh_flip =x_train[:, ::-1, ::-1, :]

#x_train_dup = np.concatenate([x_train,x_train_horizontal_flip,x_train_vertical_flip,x_train_vh_flip],axis=0)
x_train_dup = np.concatenate([x_train,x_train_horizontal_flip],axis=0)
#y_train_dup = np.concatenate([y_train, y_train,y_train, y_train], axis=0)
y_train_dup = np.concatenate([y_train, y_train], axis=0)

x_train_bis = x_train_dup.copy()
x_train_bis_bis = x_train_dup.copy()

#desplazamientos right and left 
x_train_bis[:,:,1:,:] =x_train_dup[:,:,:-1,:]
x_train_bis_bis[:,:,:-1,:] =x_train_dup[:,:,1:,:]

x_train_dup = np.concatenate([x_train_dup, x_train_bis, x_train_bis_bis],axis=0)
y_train_dup = np.concatenate([y_train_dup, y_train_dup, y_train_dup], axis=0)


In [244]:
print(f'Original dataset size: {len(x_train)}')
print(f'Augmented dataset size: {len(x_train_dup)}')

Original dataset size: 45000
Augmented dataset size: 270000


In [245]:
x_train_dup.shape, y_train_dup.shape

((270000, 32, 32, 3), (270000,))

In [246]:
from datetime import datetime
import keras

In [258]:
def create_model(input_shape=(32, 32, 3),l2_lambda=0.001):
    # Input layer for images
    inputs = layers.Input(shape=input_shape)
    
    # Flattening layer
    x = layers.Flatten()(inputs)
    
    # Normalization layer
    x = layers.Lambda(lambda x: x/np.float16(255.0))(x)
    
    # Dense layers with L2 regularization, dropout, and batch normalization
    n = 10
    for i in range(6):
        x = Dense(2**n)(x)#,
                  #kernel_regularizer=regularizers.L2(l2_lambda))(x)
        x = BatchNormalization()(x)
        x = PReLU()(x)
        x = Dropout(0.3)(x)
    
    # Output layer with softmax for 100 classes
    fine_output = layers.Dense(100, activation='softmax', name='fine_output')(x)   
  
    # Create model
    model = models.Model(inputs=inputs, outputs=[fine_output], name=f"mlp_allnighter")
    
    return model

print(f"Success at {datetime.now().time()}")

Success at 21:57:57.133776


In [259]:
model = create_model()
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={'fine_output': 'sparse_categorical_crossentropy'},
    metrics={'fine_output': 'accuracy' }
             )

# Display the model summary
model.summary()

Model: "mlp_allnighter"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_36 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_36 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_36 (Lambda)              │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_140 (Dense)               │ (None, 1024)           │     3,146,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_140         │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_21 (PReLU)              │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_80 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_141 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_141         │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_22 (PReLU)              │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_81 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_142 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_142         │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_23 (PReLU)              │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_82 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_143 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_143         │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_24 (PReLU)              │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_83 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_144 (Dense)               │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_144         │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_25 (PReLU)              │ (None, 1024)           │         1,024 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 8,527,972 (32.53 MB)

 Trainable params: 8,515,684 (32.48 MB)

 Non-trainable params: 12,288 (48.00 KB)

In [260]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True,verbose= 1)
tb = TensorBoard(log_dir="logs")
rlrop = ReduceLROnPlateau(
    monitor="val_accuracy",  # Monitor fine output validation accuracy
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-5)
callbacks = [early_stopping, tb, rlrop]

In [261]:
# Fit the model
history = model.fit(x=x_train_dup,   
                    y={'fine_output': y_train_dup},
                    validation_data=(x_val, {'fine_output': y_val}),
                    epochs=50,                        
                    batch_size=512,  # approximately 1000 batches per epoch                    
                    callbacks=callbacks)

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.0388 - loss: 4.6207 - val_accuracy: 0.1346 - val_loss: 3.7131 - learning_rate: 1.0000e-04
Epoch 2/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1176 - loss: 3.7820 - val_accuracy: 0.1690 - val_loss: 3.4884 - learning_rate: 1.0000e-04
Epoch 3/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1600 - loss: 3.5206 - val_accuracy: 0.2032 - val_loss: 3.3061 - learning_rate: 1.0000e-04
Epoch 4/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1891 - loss: 3.3470 - val_accuracy: 0.1932 - val_loss: 3.3460 - learning_rate: 1.0000e-04
Epoch 5/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2150 - loss: 3.2071 - val_accuracy: 0.2354 - val_loss: 3.1463 - learning_rate: 1.0000e-04
Epoch 6/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.2346 - loss: 3.1000 - val_accuracy: 0.2434 - val_loss: 3.1168 - learning_rate: 1.0000e-04
Epoch 7/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy

In [262]:
# Save the model
model.save("allnighter_3904.keras")

In [263]:
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [264]:
df

,Label
0,49
1,42
2,15
3,15
4,71
...,...
9995,14
9996,21
9997,18
9998,80


In [266]:
df.index.name = "Id"

In [267]:
df.to_csv("submission_39vac.csv")